In [46]:
import numpy as np
import pandas as pd

In [47]:
rating = pd.read_csv(r'C:\Users\roszs\1PyFolder\the-movies-dataset\ratings_small.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [48]:
rating = rating.drop(['timestamp'], axis = 1)

In [49]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100004 non-null  int64  
 1   movieId  100004 non-null  int64  
 2   rating   100004 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [50]:
movies = pd.read_csv(r'C:\Users\roszs\1PyFolder\the-movies-dataset\movies_metadata_fixed.csv')
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [51]:
usecols = ['id', 'title']
movies = movies[usecols]
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      45463 non-null  int64 
 1   title   45463 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.5+ KB


In [52]:
info = pd.merge(rating, movies, left_on='movieId', right_on='id').sort_values('userId')

In [53]:
info = info.drop(['id'], axis = 1)

In [54]:
info.head()

,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
182,1,2294,2.0,Jay and Silent Bob Strike Back
235,1,2455,2.5,Confidentially Yours
47,1,1405,1.0,Greed
140,1,2193,2.0,My Tutor


In [55]:
movies_ids = info['movieId'].unique()

def scale_movie_id(movie_id):
    scaled = np.where(movies_ids == movie_id)[0][0] + 1
    return scaled

info['movieId'] = info['movieId'].apply(scale_movie_id)

user_ids = info['userId'].unique()

def scale_user_id(user_id):
    scaled = np.where(user_ids == user_id)[0][0] + 1
    return scaled

info['userId'] = info['userId'].apply(scale_user_id)

n_movies = len(info['movieId'].unique())
n_users = len(info['userId'].unique())

info.head()

,userId,movieId,rating,title
0,1,1,2.5,Rocky III
182,1,2,2.0,Jay and Silent Bob Strike Back
235,1,3,2.5,Confidentially Yours
47,1,4,1.0,Greed
140,1,5,2.0,My Tutor


In [56]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(info, test_size = 0.3)

In [57]:
from sklearn.metrics import mean_squared_error 
from math import sqrt

def rmse(prediction, ground_truth):
    prediction = np.nan_to_num(prediction)[ground_truth.nonzero()].flatten()
    ground_truth = np.nan_to_num(ground_truth)[ground_truth.nonzero()].flatten()
    
    mse = mean_squared_error(prediction, ground_truth)
    return sqrt(mse)

In [58]:
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.itertuples():
    train_data_matrix[line[1] - 1, line[2] - 1] = line[3]
    
test_data_matrix = np.zeros((n_users, n_movies))
for line in test_data.itertuples():
    test_data_matrix[line[1] - 1, line[2] - 1] = line[3]

In [59]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(train_data_matrix, metric = 'cosine')  #расстояние между строками
item_similarity = pairwise_distances(train_data_matrix.T, metric = 'cosine') #между столбцами

In [64]:
def k_fract_mean_predict(top):
    top_similar = np.zeros((n_users, top))

    for i in range(n_users):
        user_sim = user_similarity[i]
        top_sim_users = user_sim.argsort()[1:top + 1]
        top_similar[i] = top_sim_users
    
    pred = np.zeros((n_users, n_movies))
    
    for i in range(n_users):
        indexes = top_similar[i].astype(np.int)
        numerator = user_similarity[i][indexes]
        mean_rating = np.array([x for x in train_data_matrix[i] if x > 0]).mean()
        diff_rating = train_data_matrix[indexes] - train_data_matrix[indexes].mean()
        product = np.dot(numerator, diff_rating)
        denominator = numerator.sum()
        pred[i] = mean_rating + product / denominator
    return pred

In [65]:
def k_fract_mean_predict_item(top):
    top_similar = np.zeros((n_movies, top))
    
    for i in range(n_movies):
        movie_sim = item_similarity[i]
        top_sim_movies = movie_sim.argsort()[1:top + 1]
        top_similar[i] = top_sim_movies
        
    pred = np.zeros((n_movies, n_users))
    
    for i in range(n_movies):
        indexes = top_similar[i].astype(np.int)
        numerator = item_similarity[i][indexes]
        mean_rating = np.array([x for x in train_data_matrix.T[i] if x > 0]).mean()
        diff_ratings = train_data_matrix.T[indexes] - train_data_matrix.T[indexes].mean()
        product = np.dot(numerator, diff_ratings)
        denominator = numerator.sum()
        pred[i] = mean_rating + product / denominator
    return pred.T

In [66]:
k_predict = k_fract_mean_predict(7)

<ipython-input-64-53067a9bf285>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indexes = top_similar[i].astype(np.int)


In [67]:
k_predict_item = k_fract_mean_predict_item(7)

<ipython-input-65-6a736943800b>:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indexes = top_similar[i].astype(np.int)
<ipython-input-65-6a736943800b>:14: RuntimeWarning: Mean of empty slice.
  mean_rating = np.array([x for x in train_data_matrix.T[i] if x > 0]).mean()
C:\Users\roszs\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-65-6a736943800b>:18: RuntimeWarning: invalid value encountered in true_divide
  pred[i] = mean_r

In [72]:
rmse(k_predict, test_data_matrix)

1.3885541949408322

In [73]:
rmse(k_predict_item, test_data_matrix)

1.4098447641782772